# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('main_task_new.csv') #Загрузка данных

In [3]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [4]:
#приведение названий столбцов к единому виду
df = df.rename(columns=lambda x: x.lower())
df = df.rename(columns=lambda x: x.replace(' ', '_'))
df.columns

Index(['restaurant_id', 'city', 'cuisine_style', 'ranking', 'rating',
       'price_range', 'number_of_reviews', 'reviews', 'url_ta', 'id_ta'],
      dtype='object')

In [5]:
#Заполнение пропусков Количество отзывов на среднее значение
df.number_of_reviews = df.number_of_reviews.fillna(df.number_of_reviews.mean())

In [6]:
#Разбиение городов на признаки
df = pd.get_dummies(df, columns=[ 'city',], dummy_na=True)

In [7]:
#Разделение кухонь в ресторанах, удаление лишних символов
df['cuisine_style'] = df['cuisine_style'].str.split(',')
df = df.explode('cuisine_style')
df.cuisine_style = df.cuisine_style.str.replace('[', '')
df.cuisine_style = df.cuisine_style.str.replace(']', '')
df.cuisine_style = df.cuisine_style.str.replace(' ', '')

<ipython-input-7-9492a4b5f1bd>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.cuisine_style = df.cuisine_style.str.replace('[', '')
<ipython-input-7-9492a4b5f1bd>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.cuisine_style = df.cuisine_style.str.replace(']', '')


In [8]:
#Разбиение кухонь на признаки
df = pd.get_dummies(df, columns=[ 'cuisine_style',], dummy_na=True)

In [9]:
#Приведение прайса к числовому формату
df.price_range = df.price_range.str.replace('$', '1')
df.price_range = df.price_range.str.replace('11 - 111', '2')
df.price_range = df.price_range.str.replace('1111', '3')
df.price_range = df.price_range.fillna(0)

<ipython-input-9-e88774da43d6>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.price_range = df.price_range.str.replace('$', '1')


In [10]:
#Перевод колонки ранжирования цен в числовой формат
df['price_range'] = df['price_range'].astype(str).astype(int)
#Удаление лишнего символа из ID ресторана в TripAdvisor и перевод в числовой формат
df.id_ta = df.id_ta.str.replace('d', '')
df['id_ta'] = df['id_ta'].astype(str).astype(int)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104896 entries, 0 to 39999
Columns: 166 entries, restaurant_id to cuisine_style_nan
dtypes: float64(3), int32(2), object(3), uint8(158)
memory usage: 22.2+ MB


In [12]:
#Не сработала
#df = [s for s in df.columns if df[s].dtypes == 'object'] 
#df.drop(df, axis = 1, inplace=True)

In [13]:
#Удаление отзывов и страниц ресторанов на 'www.tripadvisor.com'
df = df.drop(['reviews', 'url_ta'], axis = 1)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [14]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['restaurant_id', 'rating'], axis = 1)
y = df['rating']

In [15]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [16]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [17]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [18]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [19]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.09729350976204998
